In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import pandas as pd
from skimage.io import imread
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import save_model, load_model
import seaborn as sns
sns.set_theme()

path_data = 'data/'
path_train='Sentinel2LULC_354/'
path_test='Test 2/'
#path_models='gdrive/MineriaIII/models/'
path_models='drive/MyDrive/MineriaIII/models/'
#path_pred='gdrive/MineriaIII/predictions/'
path_pred='drive/MyDrive/MineriaIII/predictions/'
ext_sub = '.csv'

In [ ]:
train_zip = 'drive/MyDrive/MineriaIII/Sentinel2LULC_354.zip'
test_zip = 'drive/MyDrive/MineriaIII/Test2.zip'

!mkdir -p data
!unzip $train_zip -d $path_data
!unzip $test_zip -d $path_data

In [ ]:
test_data = []
test_labels = []
test_ids = []

for f in os.listdir(path_data + path_test):
    lab = int(re.search('^(\d{1,2})_', f).group(1))
    try:
        img = imread(path_data + path_test + f)
        test_data.append(img)
        test_labels.append(lab)
        test_ids.append(f)
    except:
        print("Cannot read file: " + f)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
label_groups = {1:1, 2:2, 3:3, 4:4,5:4, 6:5,7:5,8:5,9:5,10:5,11:5,12:5,13:5,14:5,15:5,16:5,17:5, 
                18:6,19:6,20:6, 21:7,22:7, 23:8, 24:9,25:9,26:9,27:9,28:9, 29:10}

In [ ]:
def ensemble_subgroup(model_name, subgroup):
  test_subset = []
  index_subset = []
  label_subset = []

  for i,lab in enumerate(labels):
      if label_groups.get(lab) == subgroup:
          try:
              img = imread(path_data + path_test + ids[i])
              test_subset.append(img)
              label_subset.append(lab)
              index_subset.append(i)
          except:
              print("Cannot read file: " + ids[i])

  test_subset = np.array(test_subset)
  test_subset_prep = tf.keras.applications.vgg19.preprocess_input(test_subset)

  model = load_model(path_models + model_name)
  pred_subset = np.argmax(model.predict(test_subset_prep), axis=-1) + np.min(label_subset)

  for j,ind in enumerate(index_subset):
      labels[ind] = pred_subset[j]


def predictions_ensemble(file='ensemble'):
  output = pd.DataFrame({'id.jpg': ids, 'label': labels})
  output.to_csv(path_pred + file + '.csv', header=True, index=False)

In [ ]:
predictions = pd.read_csv(path_pred + 'ensemble_waterpcasvm.csv')
ids = np.array(predictions['id.jpg'])
labels = np.array(predictions['label'])

ensemble_subgroup('crop_512-128_ft_v5.h5', 9)

predictions_ensemble('ensemble_crop_512-128_ft_v5')